# 1. User Reviews via Steam API (https://partner.steamgames.com/doc/store/getreviews)

In [1]:
# import packages
import os
import sys
import time
import json
import numpy as np
import urllib.parse
import urllib.request
from tqdm import tqdm
import plotly.express as px
from datetime import datetime
from googletrans import Translator


import pandas as pd
from pandas import json_normalize


# list package ver. etc.
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)
print('---------------')

Python version
3.8.3 (default, May 19 2020, 18:47:26) 
[GCC 7.3.0]
Version info.
sys.version_info(major=3, minor=8, micro=3, releaselevel='final', serial=0)
---------------


---
### Data Dictionary:

- Response:
    - success - 1 if the query was successful
    - query_summary - Returned in the first request
        - recommendationid - The unique id of the recommendation
        - author
            - steamid - the user’s SteamID
            - um_games_owned - number of games owned by the user
            - num_reviews - number of reviews written by the user
            - playtime_forever - lifetime playtime tracked in this app
            - playtime_last_two_weeks - playtime tracked in the past two weeks for this app
            - playtime_at_review - playtime when the review was written
            - last_played - time for when the user last played
        - language - language the user indicated when authoring the review
        - review - text of written review
        - timestamp_created - date the review was created (unix timestamp)
        - timestamp_updated - date the review was last updated (unix timestamp)
        - voted_up - true means it was a positive recommendation
        - votes_up - the number of users that found this review helpful
        - votes_funny - the number of users that found this review funny
        - weighted_vote_score - helpfulness score
        - comment_count - number of comments posted on this review
        - steam_purchase - true if the user purchased the game on Steam
        - received_for_free - true if the user checked a box saying they got the app for free
        - written_during_early_access - true if the user posted this review while the game was in Early Access
        - developer_response - text of the developer response, if any
        - timestamp_dev_responded - Unix timestamp of when the developer responded, if applicable

---
Source: https://partner.steamgames.com/doc/store/getreviews

## 1.1 Import

In [2]:
# generate game review df

#steam 'chunks' their json files (the game reviews) in sets of 100
#ending with a signature, a 'cursor'. This cursor is then pasted
#onto the the same url, to 'grab' the next chunk and so on. 
#This sequence block with an 'end cursor' of 'AoJ4tey90tECcbOXSw=='

#set variables
url_base = 'https://store.steampowered.com/appreviews/393380?json=1&filter=updated&language=all&review_type=all&purchase_type=all&num_per_page=100&cursor='

#first pass
url = urllib.request.urlopen("https://store.steampowered.com/appreviews/393380?json=1&filter=updated&language=all&review_type=all&purchase_type=all&num_per_page=100&cursor=*")
data = json.loads(url.read().decode())
next_cursor = data['cursor']
next_cursor = next_cursor.replace('+', '%2B')
df1 = json_normalize(data['reviews'])
print(next_cursor)

#add results till stopcursor met, then send all results to csv
while True:
    time.sleep(0.5) # Sleep for one second
    url_temp = url_base + next_cursor
    url = urllib.request.urlopen(url_temp)
    data = json.loads(url.read().decode())
    next_cursor = data['cursor']
    next_cursor = next_cursor.replace('+', '%2B')
    df2 = json_normalize(data['reviews'])
    df1 = pd.concat([df1, df2])
    print(next_cursor)
    if next_cursor == 'AoJ44PCp0tECd4WXSw==' or next_cursor == '*':
        df_steam_reviews = df1
        df1 = None
        break
        
#the hash below is each 'cursor' I loop through until the 'end cursor'.
#this is just my way to monitor the download.

AoJwhdbU3fICeIqrjgI=
AoJ4%2BYqm1/ICfNHojQI=
AoJwtZW10fICf5mzjQI=
AoJ4yKGsy/ICeYf/jAI=
AoJwzvaoxfICdaHGjAI=
AoJ4kfrtwPICeZ2ajAI=
AoJ4oqjQu/ICcYnsiwI=
AoJ4wNKeuPICe43NiwI=
AoJ4pbS9svICdPaUiwI=
AoJwwsOyrPICfK/bigI=
AoJ4y63nqPICcIi3igI=
AoJwgaj5o/ICc6qEigI=
AoJwtvbzn/ICe%2B3G2wE=
AoJ4mbGunPICe72tiQI=
AoJwqbfcl/ICcNn2iAI=
AoJwx/LRk/ICepLQbg==
AoJw542Mj/ICfOySiAI=
AoJw7u2QjPICeoG98AE=
AoJ4qYLth/ICcJ7EhwI=
AoJ4yofHhPICe9WghwI=
AoJ4pcnJgvICeLqNhwI=
AoJwkNDZ//ECcITwhgI=
AoJw6emx/fECfsy1pQE=
AoJ4qujk%2BvECcJy8hgI=
AoJ4oq6a%2BfECdveqhgI=
AoJ4k7fv%2BPECfNWnhgI=
AoJ4tN21%2BPECdvmihgI=
AoJwre3y9/ECefKbhgI=
AoJ4k%2BmR9vECfaCJhgI=
AoJw9Pin9fECe7j/hQI=
AoJw7K288vECcuTehQI=
AoJ4h7q08PECeKTIhQI=
AoJwjaea7/ECdLu5hQI=
AoJw/p%2Bh7vECd5%2BwhQI=
AoJw64Do7fECdsWqhQI=
AoJwy6So7fECc%2B6jhQI=
AoJw4Ln46/ECeN%2BUhQI=
AoJwsO%2BY6/ECeqaLhQI=
AoJ4o7q16vECequAhQI=
AoJ4xsy86fECdbX2hAI=
AoJ4xcDR6PECfPTrhAI=
AoJw0%2BLc5/ECfufehAI=
AoJ4mY615vECd97QhAI=
AoJwl9PQ4fECcOOYhAI=
AoJ4z5rM2/ECcMHTgwI=
AoJwnJKX1vECdNaSgwI=
AoJwu8yx

In [3]:
# inspect columns
print(df_steam_reviews.info(verbose=True))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51204 entries, 0 to 3
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   recommendationid                51204 non-null  object 
 1   language                        51204 non-null  object 
 2   review                          51204 non-null  object 
 3   timestamp_created               51204 non-null  int64  
 4   timestamp_updated               51204 non-null  int64  
 5   voted_up                        51204 non-null  bool   
 6   votes_up                        51204 non-null  int64  
 7   votes_funny                     51204 non-null  int64  
 8   weighted_vote_score             51204 non-null  object 
 9   comment_count                   51204 non-null  int64  
 10  steam_purchase                  51204 non-null  bool   
 11  received_for_free               51204 non-null  bool   
 12  written_during_early_access     5120

In [4]:
# inspect shape
print(df_steam_reviews.shape)

(51204, 21)


In [5]:
# inspect df
df_steam_reviews

,recommendationid,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,...,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.last_played,timestamp_dev_responded,developer_response
0,71001079,schinese,fuck youself,1592520117,1592520117,True,0,0,0,0,...,False,True,76561198895153543,14,1,1873,100,1592520945,NaN,NaN
1,71000946,english,in the start it was god..in the middel i was g...,1592519841,1592519841,False,0,0,0,0,...,False,True,76561198006640217,51,4,15380,1048,1592525984,NaN,NaN
2,70998547,english,Its not terrable.,1592515324,1592515324,True,0,0,0,0,...,False,True,76561198160636998,19,1,3695,46,1592498224,NaN,NaN
3,70996916,french,ce jeux est reelement imercif je kiff se type ...,1592512577,1592512577,True,0,0,0,0,...,False,True,76561198286418169,10,1,598,0,1575313560,NaN,NaN
4,70996898,russian,"ZBS игруля, не пожалел",1592512532,1592512532,True,0,0,0,0,...,False,True,76561198851917666,3,1,1553,191,1592522636,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,19708009,english,Communication and teamplay is one of the best ...,1450166529,1450166529,True,44,3,0.566995859146118164,0,...,False,True,76561198014615390,251,2,13623,0,1589089421,NaN,NaN
0,19708002,english,If you're looking for a realistic mil-sim styl...,1450166484,1450166484,True,38,2,0.63659060001373291,0,...,False,True,76561198066675517,543,9,4512,0,1587823021,NaN,NaN
1,19707993,english,ITS THE BEST GAME EVERS \n\nPlay this game if ...,1450166426,1450166426,True,205,517,0.549310982227325439,13,...,False,True,76561198048218734,93,11,112520,115,1591500969,NaN,NaN
2,19707992,english,(Disclaimer: This game used to not be on Steam...,1450166424,1450166424,True,157,9,0.790276587009429932,0,...,False,True,76561198038225005,253,8,3969,0,1589425139,NaN,NaN


In [6]:
# save that sheet
df_steam_reviews.to_csv('squad_reviews.csv', index=False)

## 1.2 Clean

In [23]:
#search for presence of empty cells
df_steam_reviews.isnull().sum(axis = 0)

AttributeError: 'list' object has no attribute 'isnull'

In [24]:
#drop empty cols 'timestamp_dev_responded' and 'developer_response'
df_steam_reviews = df_steam_reviews.drop(['timestamp_dev_responded', 'developer_response'], axis=1)

AttributeError: 'list' object has no attribute 'drop'

In [25]:
# convert unix timestamp columns to datetime format
def time_to_clean(x):
    return datetime.fromtimestamp(x)

df_steam_reviews['timestamp_created'] = df_steam_reviews['timestamp_created'].apply(time_to_clean)
df_steam_reviews['timestamp_updated'] = df_steam_reviews['timestamp_updated'].apply(time_to_clean)
df_steam_reviews['author.last_played'] = df_steam_reviews['author.last_played'].apply(time_to_clean)

TypeError: list indices must be integers or slices, not str

In [7]:
# translate/spellcheck via googletranslate pkg


# pass 'review' col as str
df_steam_reviews['review'] = df_steam_reviews['review'].astype(str)

# instantiate tqdm instance
tqdm.pandas()

def time_to_translate(x):
    time.sleep(1)
    if x == None: # ignore the 'NaN' reviews
        return 'NaN'
    else:
        translator = Translator()
        text = getattr(translator.translate(x), 'text')
        return text
    
# make empty 'review.translated' col
df_steam_reviews['review.translated'] = None

# split df_steam_reviews into 10000 pieces
# I had to `np.array_split()` my df because google translate's api kept crashing randomly
# and I couldn't keep starting over and over and over for a "perfect" notebook. 
df_steam_reviews = np.array_split(df_steam_reviews, 10000)

/home/carl/anaconda3/envs/text-as-data/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [19]:
a = pd.read_csv('a_temp.csv', low_memory=False)

In [50]:
for item in range(8001,9999):
    b = df_steam_reviews[item]
    b['review.translated'] = b['review'].progress_apply(time_to_translate)
    frames = [a, b]
    a = pd.concat(frames)

100%|██████████| 5/5 [00:08<00:00,  1.70s/it]


In [48]:
a.to_csv('a_temp.csv', index=False)

In [49]:
a

,recommendationid,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.last_played,review.translated
0,71001079,schinese,fuck youself,2020-06-18 17:41:57,2020-06-18 17:41:57,True,0,0,0.000000,0,True,False,True,76561198895153543,14,1,1873,100,2020-06-18 17:55:45,fuck yourself
1,71000946,english,in the start it was god..in the middel i was g...,2020-06-18 17:37:21,2020-06-18 17:37:21,False,0,0,0.000000,0,False,False,True,76561198006640217,51,4,15380,1048,2020-06-18 19:19:44,in the start it was god..in the middel i was g...
2,70998547,english,Its not terrable.,2020-06-18 16:22:04,2020-06-18 16:22:04,True,0,0,0.000000,0,False,False,True,76561198160636998,19,1,3695,46,2020-06-18 11:37:04,It's not terrible.
3,70996916,french,ce jeux est reelement imercif je kiff se type ...,2020-06-18 15:36:17,2020-06-18 15:36:17,True,0,0,0.000000,0,False,False,True,76561198286418169,10,1,598,0,2019-12-02 13:06:00,this game is really imercive I like this type ...
4,70996898,russian,"ZBS игруля, не пожалел",2020-06-18 15:35:32,2020-06-18 15:35:32,True,0,0,0.000000,0,True,False,True,76561198851917666,3,1,1553,191,2020-06-18 18:23:56,"ZBS play, did not regret"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41199,30802454,german,Wo bin ich hier gelandet? Altersdurchschnitt d...,2017-03-28 17:01:08,2017-03-28 17:01:08,True,0,0,0.000000,0,True,False,True,76561198003708795,161,3,12708,0,2020-04-09 19:08:36,Where have I ended up? Average age of the play...
41200,30798792,schinese,游戏好不好？看我玩的时间就知道了，我可是玩了900多个小时才来测评啊，总之不亏，注意看测评时...,2017-03-28 13:25:36,2017-03-28 13:25:36,True,79,20,0.817044,4,True,False,True,76561198009144997,117,13,181588,0,2020-05-25 09:38:54,"Is the game good? I know how long I play, but ..."
41201,30796794,swedish,"Pew pew, I like it!",2017-03-28 11:43:39,2017-03-28 11:43:39,True,0,0,0.000000,0,True,False,True,76561198138574728,189,61,1971,0,2019-10-14 11:38:39,"Pew pew, I like it!"
41202,26549515,english,Were now at V9. Game ever changing and getting...,2016-11-11 20:39:40,2017-03-28 10:57:16,True,1,0,0.485189,0,True,False,True,76561198157021215,62,3,108806,16,2020-06-13 19:10:18,Were now at V9. Game ever changing and getting...


# 2. Users Per Day (https://steamdb.info/app/393380/graphs/)

In [ ]:
#load 'steam charts for every day' downloaded by hand from steamdb on june 18th 
df_upd = pd.read_csv('users_per_day.csv')

In [ ]:
#convert 'DateTime' col to datetime Dtype.
df_upd['DateTime'] = pd.to_datetime(df_upd['DateTime'])

In [ ]:
#inspect cols
print(df_upd.info(verbose=True))

In [ ]:
#inspect shape
df_upd.shape

In [ ]:
#inspect df
df_upd

In [ ]:
#display users per day
fig = px.line(df_upd, x="DateTime", y="Players", width=1000, height=500, title='Users Per Day')
fig.show()

In [ ]:
#display twitch viewers per day
fig = px.line(df_upd, x="DateTime", y="Twitch Viewers", width=1000, height=500, title='Users Per Day')
fig.show()

# steamspy (https://steamspy.com/search.php)

In [ ]:
#need 
user retention
sales figures

# Misc

In [ ]:
# list of free weekends:
Squad Free Weekend - Nov 2016
Squad Free Weekend - Apr 2017
Squad Free Weekend - Nov 2017
Squad Free Weekend - Jun 2018
Squad Free Weekend - Nov 2018
Squad Free Weekend - Jul 2019
Squad Free Weekend - Nov 2019

# list of major patch days:
v1 - July 1 2015
v2 - Oct 31 2015
v3 - Dec 15 2015
v4 - ?
v5 - Mar 30 2016
v6 - May 26 2016
v7 - Aug  7 2016
v8 - Nov  1 2016
v9 - Mar  9 2017
v10  Feb  5 2018
v11  Jun  6 2018
v12  Nov 29 2018
v13  May  ? 2019
v14  Jun 28 2019
v15  Jul 22 2019
v16  Oct 10 2019
v17  Nov 25 2019
v18  ?
v19  May  2 2020

![Rating-Formula](https://steamdb.info/static/img/blog/84/formula.png)

In [ ]:
#v2 (fromhttps://cloud.google.com/translate/docs/simple-translate-call#translate_translate_text-python)
# translate/spellcheck via googletranslate pkg
from google.cloud import translate_v2 as translate

def time_to_translate(x):
    if x == None: # ignore the 'NaN' reviews
        return 'NaN'
    else:
        translate_client = translate.Client()
        if isinstance(x, six.binary_type):
            text = x.decode('utf-8')
            return text
        
#print(time_to_translate('hola'))

In [3]:
# scratch
df_steam_reviews = pd.read_csv('squad_reviews.csv', low_memory=False)

In [ ]:
df_steam_reviews

In [ ]:
# display reviews 
fig = px.histogram(df_steam_reviews, x="timestamp_created", color="voted_up", width=1000, height=500, title='Positive(True)/Negative(False) Reviews')
fig.show()

In [ ]:
# translate/spellcheck t
t['review.translated'] = t['review'].progress_apply(time_to_translate)
t.to_csv('t.csv', index=False)